# Geospatial Join and Visualization Tutorial #
This is a tutorial to show you how to join your geospatial data to polygon geometry and visualize it in Carto.

start with a geospatial-enabled cluster with Apache Sedona installed.

In the upper right hand corner, select "Connect" and select "geospatial-cluster-smallest-available". It may take a few minutes to start.

In [0]:
%pip install folium

In [0]:
dbutils.library.restartPython()

In [0]:
import sedona
import geopandas as gpd
import fiona

Go to the catalog to find the path to your data set. They are stored as Delta Tables, a variant of Parquety which are light-weight, static files on S3, which is easy to query and has version control features.

## read the table ##

In [0]:
sdf = spark.read.table("moo_ops_workspace.threeoneone.sr_mirror_2024")
sdf.printSchema()
sdf.display(5)

## import tools 

In [0]:
from pyspark.sql.functions import col, expr

##Parse the Lat Lon into a point##  

Carto stores everything as a WKB or Well-Known Binary. To join two geometries they need to both be in the same WKB Format. 

You may convert to WKB using ST_AsBinary(). This is called serialization. This converts it into a from that Carto can read.

You may reverse this by ST_GeomFromWKB. This is called deserialization. This converts it into a format that Databricks can read.

'ST' Either stands for Spatial Type or 'Spatial Transformation', not sure. When you cast the Latitudes and Longitudes using ST_Point, your Spark Dataframe becomes a Sedona Geospatial dataframe.

In [0]:
#turn your spark dataframe into a sedona geodataframe by creating an ST_POINT colum.
points_311 = sdf.withColumn("the_geom", expr(f"ST_AsBinary(ST_Point(LON, LAT))"))
points_311 = points_311.withColumn("geom_deserialized", expr(f"ST_GeomFromWKB(the_geom)"))
points_311.display(5)

## Import a geospatial dataset ##
## In this step we import a Carto-Prepared geometry. ##
Sedona is already registered in the geospatial clusters.

When you attempt to import a map into Carto, it will throw an error and ask you if you want to prepare it. A *Prepared map* adds the carto max and min bounding box to the dataframe.

# Convert a Shapefile into a Sedona dataframe #

In [0]:
shapefile_filepath = "/Volumes/moo_ops_workspace/geospatial/geospatial_files_volume/BusinessImprovementDistrict.zip"
#if you import a shapefile, you have to specify the schema. We will use a geojson instead.
filepath = "/Volumes/moo_ops_workspace/geospatial/geospatial_files_volume/BusinessImprovementDistrict.geojson"
filepath_no_crs = "/Volumes/moo_ops_workspace/geospatial/geospatial_files_volume/BusinessImprovementDistrict_no_crs.geojson"


In [0]:

gdf = gpd.read_file(filepath)
gdf.info()

In [0]:
#read the shapefile as a sedona dataframe

bid_map = spark.read.format("shapefile").load(shapefile_filepath)
#bid_map = spark.read.format("geojson").load(filepath_no_crs)
bid_map.display(5)

In [0]:
with fiona.open(shapefile_filepath, 'r') as shapefile:
    print(shapefile.schema)

In [0]:
bid_map = bid_map.withColumn("geom_deserialized", expr(f"ST_GeomFromWKB(the_geom)"))
bid_map = bid_map.display(5)
bid_map.printSchema()

## Join the two maps using geospatial SQL

the point geometry will be suprerceded with the polygon geometry

In [0]:
#pass the data frames in as variables in an python f-string
points_311.createOrReplaceTempView("points_311")
nypd_map.createOrReplaceTempView("nypd_map")
#group by all the geometry columns to keep them.
#In order to keep the map carto-prepared you need to preserve the __carto_ columns.
result_df = spark.sql(f"""SELECT
                        FIRST(__carto_xmin) AS __carto_xmin,
                        FIRST(__carto_xmax) AS __carto_xmax, 
                        FIRST(__carto_ymin) AS __carto_ymin, 
                        FIRST(__carto_ymax) AS __carto_ymax, 
                        COUNT(*) as 311_calls, 
                        nypd_map.precinct,
                        nypd_map.the_geom,
                        nypd_map.geom_deserialized
                        
                      FROM points_311, nypd_map
                      --ST_Contains is where the spatial join happens.
                      WHERE ST_Contains(nypd_map.geom_deserialized, points_311.geom_deserialized)
                      GROUP BY nypd_map.geom_deserialized, nypd_map.precinct, nypd_map.the_geom
                      """)
result_df.display()


# Save the Map#

In [0]:
%sql
DROP TABLE IF EXISTS moo_ops_workspace.threeoneone.311_calls_by_precinct

In [0]:

#save result_df as a geospatial delta table in the catalog
result_df.write.format("delta").mode("overwrite").saveAsTable("moo_ops_workspace.threeoneone.311_calls_by_precinct")

In [0]:
#save as a non-geospatial table
no_map_df = result_df[['311_calls', 'precinct']]
no_map_df.write.format('delta').mode("overwrite").saveAsTable("moo_ops_workspace.threeoneone.311_calls_no_map")

# Visualization in PyDeck #

Display using pyDeck

This isn't working for me.

In [0]:
# Define a function to map 311_calls to colors
def get_color(calls):
    if calls < 20000:
        return [255, 255, 204, 160]
    elif calls < 40000:
        return [255, 237, 160, 160]
    elif calls < 60000:
        return [254, 217, 118, 160]
    elif calls < 80000:
        return [254, 178, 76, 160]
    else:
        return [253, 141, 60, 160]

In [0]:
def get_color_rgb(calls):
    if calls < 20000:
        return "rgb(255, 255, 204)"
    elif calls < 40000:
        return "rgb(255, 237, 160)"
    elif calls < 60000:
        return "rgb(254, 217, 118)"
    elif calls < 80000:
        return "rgb(254, 178, 76)"
    else:
        return "rgb(253, 141, 60)"

In [0]:
import pydeck as pdk
import pandas as pd
import geopandas as gpd
import shapely
import sys
sys.setrecursionlimit(100000)


In [0]:
# have to get out of spark dataframe and into a geopandas geodataframe.
df = result_df.toPandas()

#load the geometry into a geopandas dataframe
#df['geometry'] = df['the_geom'].apply(shapely.wkb.loads)
df['geometry'] = df['geom_deserialized']
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
gdf.head(5)
#apply a fill color for the data
gdf['fill_color'] = gdf['311_calls'].apply(get_color)
gdf['fill_color_rgb'] = gdf['311_calls'].apply(get_color_rgb)
del gdf['the_geom']
del gdf['geom_deserialized']
#simplify the geometry to make it smaller. Prevents the json recursion from getting too deep.
gdf['geometry'] = gdf['geometry'].simplify(0.0001)
geojson = gdf.to_json()


In [0]:
gdf.head(5)
geojson[:1000]

In [0]:

LAND_COVER = [[-74.2654, 40.4964], [-73.7004, 40.4964], [-73.7004, 40.9156], [-74.2654, 40.9156]]
geojson_layer = pdk.Layer(
    "GeoJsonLayer",
    geojson,
    opacity=0.8,
    stroked=True, 
    filled=True,
    get_fill_color='properties.fill_color',
    pickable=True)
'''
polygon_layer = pdk.Layer(
    "PolygonLayer",
    LAND_COVER,
    get_polygon='-',
    get_fill_color='[0, 0, 0, 20]',)
'''
view_state = pdk.ViewState(latitude=40.7128, longitude=-74.0060, zoom=11, bearing=0, pitch=0)

r = pdk.Deck(layers=[geojson_layer], 
             initial_view_state=view_state)
r.to_html()

# Visualization using Folium #

Was able to get this to work. Folium is widely used in industry for map visualization.

In [0]:
import folium

In [0]:
gdf['fill_color_rgb'][:5]

In [0]:
geojson = gdf.to_json()

In [0]:
m = folium.Map(location=[40.7128, -74.0060], zoom_start=11)
# folium takes color properties in the format "rgb(255, 255, 255)" or "rgba(255, 255, 255, 1.0)". It also accepts hex "#ff0000" for color
folium.GeoJson(geojson, 
               style_function=lambda feature: { 'fillColor': feature['properties']['fill_color_rgb'],
                                                'color': 'black', 
                                                'weight': 1, 
                                                'fillOpacity': 0.8
                                             }           
               ).add_to(m)
display(m)